In [1]:
import pandas as pd 
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import CountVectorizer   
from sklearn.preprocessing import LabelEncoder
from sklearn.mixture import GaussianMixture
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score, auc, roc_curve, roc_auc_score

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df = pd.read_csv("/content/drive/My Drive/FullDataset.csv")
df.iloc[1,:]


Unnamed: 0                                                      1
title                    "Strengthening accountability to end TB"
text                                                          NaN
source                                                        NaN
label                                                           1
title_text      strengthening accountability  end tb  today re...
type                                                      article
newstitle       "WHO offers national assessment checklist for ...
publish_date                                                  NaN
content         who today released a checklist to enable membe...
Name: 1, dtype: object

In [4]:
df.isnull().sum()

Unnamed: 0         0
title            446
text            3919
source          3939
label              0
title_text         0
type            1568
newstitle       2007
publish_date    4334
content         1985
dtype: int64

In [5]:
for i in range(0,len(df["content"])):
    if type(df.iloc[i,-1]) == float:
        (df.iloc[i,-1])= "0" 
for i in range(0,len(df["newstitle"])):
    if type(df.iloc[i,-3]) == float:
        (df.iloc[i,-3])= "0"        
for i in range(0,len(df["title"])):
    if type(df.iloc[i,1]) == float:
        (df.iloc[i,1])= "0"
for i in range(0,len(df["text"])):
    if type(df.iloc[i,2]) == float:
        (df.iloc[i,2])= "0"
df['merge']=df['title'].str.cat(df['content'],sep=" ")
df['merge']=df['newstitle'].str.cat(df['merge'],sep=" ")
df['merge']=df['title_text'].str.cat(df['merge'],sep=" ")
df

,Unnamed: 0,title,text,source,label,title_text,type,newstitle,publish_date,content,merge
0,0,ÒItÕs actually the safest time to fly.Ó,0,NaN,0,actually safest time fly thousands flights...,article,"""Photos of deserted, nearly empty airports aro...",2020-03-01T18:39:00Z,thousands of flights around the world have bee...,actually safest time fly thousands flights...
1,1,"""Strengthening accountability to end TB""",0,NaN,1,strengthening accountability end tb today re...,article,"""WHO offers national assessment checklist for ...",NaN,who today released a checklist to enable membe...,strengthening accountability end tb today re...
2,2,Lack of Paid Sick Leave a Coronavirus Threat,0,NaN,1,lack paid sick leave coronavirus threat edit...,article,"""Lack of Paid Sick Leave a Coronavirus Threat""",NaN,editor 's note for the latest updates on the 2...,lack paid sick leave coronavirus threat edit...
3,3,States are imposing mandatory leave and statew...,0,NaN,0,states imposing mandatory leave statewise sc...,post,0,NaN,0,states imposing mandatory leave statewise sc...
4,4,Guidance for Administrators in Parks and Recre...,0,NaN,1,guidance administrators parks recreational ...,article,"""Information for Laboratories about Coronaviru...",2/11/20,these sequences are intended to be used for th...,guidance administrators parks recreational ...
...,...,...,...,...,...,...,...,...,...,...,...
5078,5078,“The mainstream media pretended there was a de...,0,NaN,0,mainstream media pretended deadly surge c...,post,"""What the MSM Was Afraid to Tell You""",NaN,this week while the mainstream mainstream medi...,mainstream media pretended deadly surge c...
5079,5079,Repercussions From Antibiotics Overuse During ...,0,NaN,1,repercussions antibiotics overuse pandemic ...,article,"""Access denied | www.webmd.com used Cloudflare...",NaN,this website is using a security service to pr...,repercussions antibiotics overuse pandemic ...
5080,5080,Study Confirms Cats Can Become Infected With a...,0,NaN,1,study confirms cats become infected may tra...,article,"""""",NaN,0,study confirms cats become infected may tra...
5081,5081,Texas Gov. Pauses Economic Reopening,0,NaN,1,texas gov pauses economic reopening website ...,article,"""Access denied | www.webmd.com used Cloudflare...",NaN,this website is using a security service to pr...,texas gov pauses economic reopening website ...


In [6]:
import re
import string
 
df1=df
df1['merge'] = df1['merge'].str.replace('[^a-zA-Z \n\.]',"")
df1['merge'] = df1['merge'].str.lower()
df1['merge']=df1['merge'].apply(str)
#df1['merge'] = re.sub(r'\d+', '',df1['merge']) # remove numbers
#df1['merge'] = df1['merge'].translate(string.maketrans("",""), string.punctuation) #remove punctuations

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  """


In [7]:
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [8]:
X = input("Enter first string: ").lower() 
sim=[]
index=[]
 
for i in range(0,len(df1["merge"])):
    Y =df1["merge"][i]
    #X ="cover like flu and you can cure by vitamine C"
    # tokenization 
    X_list = word_tokenize(X)  
    Y_list = word_tokenize(Y) 
      
    # sw contains the list of stopwords 
    sw = stopwords.words('english')  
    a =[];b =[] 
      
    # remove stop words from the string 
    X_set = {w for w in X_list if not w in sw}  
    Y_set = {w for w in Y_list if not w in sw} 
      
    # form a set containing keywords of both strings  
    rvector = X_set.union(Y_set)  
    for w in rvector: 
        if w in X_set:
          a.append(1) # create a vector 
        else: 
          a.append(0) 
        if w in Y_set:
          b.append(1) 
        else: 
          b.append(0)
    c = 0
      
    # cosine formula  
    for j in range(len(rvector)): 
            c+= a[j]*b[j] 
    cosine = c / float((sum(a)*sum(b))**0.5) 
    sim.append(cosine)
    index.append(i)
d = {'Score':sim,'Index':index}
df3 = pd.DataFrame(d)
df3=df3.sort_values(by='Score', ascending=False)
df3.head()

Enter first string: COVID19 needs vitamin C to be cured.


,Score,Index
2694,0.500000,2694
3996,0.369274,3996
2921,0.369274,2921
3753,0.339683,3753
4213,0.339683,4213


In [9]:
if df.iloc[int(df3.iloc[0][1]),9] != "0":
     print(df.iloc[int(df3.iloc[0][1]),9])
     if df.iloc[int(df3.iloc[0][1]),4]==1:
                  print("\n real news")
     else:
                  print("\n real news") 
elif  df.iloc[int(df3.iloc[0][1]),2] != "0":
    print(df.iloc[int(df3.iloc[0][1]),2])
    if df.iloc[int(df3.iloc[0][1]),4]==1:
                  print("\n real news")
    else:
                  print("\n real news") 
elif df.iloc[int(df3.iloc[0][1]),1] != "0":
    print(df.iloc[int(df3.iloc[0][1]),1])
    if df.iloc[int(df3.iloc[0][1]),4]==1:
                  print("\n real news")
    else:
                  print("\n real news")
 
elif df.iloc[int(df3.iloc[0][1]),7] != "0":
    print(df.iloc[int(df3.iloc[0][1]),7])
    if df.iloc[int(df3.iloc[0][1]),4]==1:
                  print("\n real news")
    else:
                  print("\n real news")

Vitamin C can ÒstopÓ the new coronavirus.

 real news
